In [1]:
import numpy as np
import pandas as pd
np.set_printoptions(threshold=123456789)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

import math, random
from collections import Counter
from itertools import product
from copy import deepcopy

import os

In [2]:
def read_dti_data(pwd = '/home/chujunyi/2_Program/2_output_file/1_construct_dataset/3_drop_repeat_smiles_from2folder/', target = 'GPCR'):
    dti_data = pd.read_csv(pwd + 'v2_' + target + '_updated_drug_smiles_ids_drop_repeated.csv')[['drug_id', 'hsa_id']]
    print('DTIs shape = ', dti_data.shape)
    return dti_data

In [3]:
def impute_feature_0(dti_data, feat_data):
    try:
        diff_hsa = list(set(dti_data['hsa_id']).difference(set(feat_data['Query']))) # pfam
    except:
        diff_hsa = list(set(dti_data['hsa_id']).difference(set(feat_data['Feature']))) # profeat
    print(diff_hsa)
    
    for hsa in diff_hsa:
        feat_data.loc[len(feat_data)] = [hsa] + [0] * (feat_data.shape[1] - 1) 
    try:
        assert list(set(dti_data['hsa_id']).difference(set(feat_data['Query']))) == []
    except:
        assert list(set(dti_data['hsa_id']).difference(set(feat_data['Feature']))) == []
    print('Impute Feature: ',feat_data.shape)
    return feat_data

In [4]:
def generate_xy_file(dti_data, feat_data, target = 'NR', ctd = False, pro = False, pfam = False):

    if ctd:
        col_name = 'Feature'
        filename = '_U_D_xy_np_profeatCTD.npz'
    elif pro:
        col_name = 'Feature'
        filename = '_U_D_xy_np_profeatPRO.npz'
    elif pfam:
        col_name = 'Query'
        filename = '_U_D_xy_np_PFAM.npz'
        
    data_drug_label = pd.concat([dti_data, pd.get_dummies(dti_data['drug_id'])], axis = 1).groupby(by = 'hsa_id').sum().reset_index()
    data_drug_label = pd.merge(feat_data, data_drug_label, left_on = col_name, right_on = 'hsa_id').sample(frac = 1.0, random_state = 1231).reset_index(drop = True)
    del data_drug_label['hsa_id']
    
    label_num = len(set(dti_data['drug_id']))
    x_drug_label = data_drug_label.iloc[:, 1:-label_num] # 1 columns: col_name
    y_drug_label = data_drug_label.iloc[:, -label_num:]
    assert x_drug_label.shape[1] == feat_data.shape[1] - 1 # 2 columns:smiles, drug_id
    assert y_drug_label.shape[1] == label_num
    
    target_ids = np.array(data_drug_label[col_name].tolist())
    drug_ids = np.array(y_drug_label.columns.tolist())
    
    x_drug_label_np = np.array(x_drug_label)
    y_drug_label_np = np.array(y_drug_label)
    
    if ctd or pro:
        norm_idx = np.array(range(x_drug_label_np.shape[1]))
    else:
        norm_idx = np.array([])
    
    print(x_drug_label_np.shape, y_drug_label_np.shape, target_ids.shape, drug_ids.shape, len(norm_idx))
    
    pwd = '/home/chujunyi/2_Program/2_output_file/2_multilabel/1_X_Y_data/'
    np.savez_compressed(pwd + target + filename, 
                        x_d_np = x_drug_label_np, 
                        y_d_np = y_drug_label_np,
                        norm_idx = norm_idx,
                        target_ids = target_ids,
                        drug_ids = drug_ids)

    return x_drug_label, y_drug_label, target_ids, drug_ids, data_drug_label

# RUN CTD

In [5]:
nr_dti = read_dti_data(target = 'nr')
nr_ctd = pd.read_csv('/home/chujunyi/2_Program/2_output_file/4_target_feature/NR_PROFEAT_CTD.csv')
nr_x_ctd, nr_y_ctd, nr_target_ctd, nr_drug_ctd, nr_ctd_data = generate_xy_file(nr_dti, nr_ctd, target = 'NR', ctd = True)

gpcr_dti = read_dti_data(target = 'GPCR')
gpcr_ctd = pd.read_csv('/home/chujunyi/2_Program/2_output_file/4_target_feature/GPCR_PROFEAT_CTD.csv')
gpcr_x_ctd, gpcr_y_ctd, gpcr_target_ctd, gpcr_drug_ctd, gpcr_ctd_data = generate_xy_file(gpcr_dti, gpcr_ctd, target = 'GPCR', ctd = True)

ic_dti = read_dti_data(target = 'IC')
ic_ctd = pd.read_csv('/home/chujunyi/2_Program/2_output_file/4_target_feature/IC_PROFEAT_CTD.csv')
ic_x_ctd, ic_y_ctd, ic_target_ctd, ic_drug_ctd, ic_ctd_data = generate_xy_file(ic_dti, ic_ctd, target = 'IC', ctd = True)

e_dti = read_dti_data(target = 'E')
e_ctd = pd.read_csv('/home/chujunyi/2_Program/2_output_file/4_target_feature/E_PROFEAT_CTD.csv')
e_ctd = impute_feature_0(e_dti, e_ctd)
e_x_ctd, e_y_ctd, e_target_ctd, e_drug_ctd, e_ctd_data = generate_xy_file(e_dti, e_ctd, target = 'E', ctd = True)

DTIs shape =  (886, 2)
(33, 504) (33, 541) (33,) (541,) 504
DTIs shape =  (5383, 2)
(156, 504) (156, 1680) (156,) (1680,) 504
DTIs shape =  (6385, 2)
(238, 504) (238, 765) (238,) (765,) 504
DTIs shape =  (7371, 2)
['hsa390956', 'hsa94009']
Impute Feature:  (1431, 505)
(1411, 504) (1411, 1777) (1411,) (1777,) 504


# RUN PRO

In [6]:
nr_dti = read_dti_data(target = 'nr')
nr_pro = pd.read_csv('/home/chujunyi/2_Program/2_output_file/4_target_feature/NR_PROFEAT.csv')
nr_x_pro, nr_y_pro, nr_target_pro, nr_drug_pro, nr_pro_data = generate_xy_file(nr_dti, nr_pro, target = 'NR', pro = True)

gpcr_dti = read_dti_data(target = 'GPCR')
gpcr_pro = pd.read_csv('/home/chujunyi/2_Program/2_output_file/4_target_feature/GPCR_PROFEAT.csv')
gpcr_x_pro, gpcr_y_pro, gpcr_target_pro, gpcr_drug_pro, gpcr_pro_data = generate_xy_file(gpcr_dti, gpcr_pro, target = 'GPCR', pro = True)

ic_dti = read_dti_data(target = 'IC')
ic_pro = pd.read_csv('/home/chujunyi/2_Program/2_output_file/4_target_feature/IC_PROFEAT.csv')
ic_x_pro, ic_y_pro, ic_target_pro, ic_drug_pro, ic_pro_data = generate_xy_file(ic_dti, ic_pro, target = 'IC', pro = True)

e_dti = read_dti_data(target = 'E')
e_pro = pd.read_csv('/home/chujunyi/2_Program/2_output_file/4_target_feature/E_PROFEAT.csv')
e_pro = impute_feature_0(e_dti, e_pro)
e_x_pro, e_y_pro, e_target_pro, e_drug_pro, e_pro_data = generate_xy_file(e_dti, e_pro, target = 'E', pro = True)

DTIs shape =  (886, 2)
(33, 1437) (33, 541) (33,) (541,) 1437
DTIs shape =  (5383, 2)
(156, 1437) (156, 1680) (156,) (1680,) 1437
DTIs shape =  (6385, 2)
(238, 1437) (238, 765) (238,) (765,) 1437
DTIs shape =  (7371, 2)
['hsa390956', 'hsa94009']
Impute Feature:  (1431, 1438)
(1411, 1437) (1411, 1777) (1411,) (1777,) 1437


# PFAM domain

In [7]:
def read_pfam(pwd = '/home/chujunyi/2_Program/2_output_file/4_target_feature/', target = 'NR'):
    data = pd.read_table(pwd + target + '_NCBI-PFAM_Full.txt', sep = '\t')[['Query', 'Accession']]
    data = pd.concat([data, pd.get_dummies(data['Accession'])], axis = 1).groupby(by = 'Query').sum().reset_index()
    print('PFAM: ', data.shape)
    return data

In [8]:
nr_dti = read_dti_data(target = 'nr')
gpcr_dti = read_dti_data(target = 'GPCR')
ic_dti = read_dti_data(target = 'IC')
e_dti = read_dti_data(target = 'E')

DTIs shape =  (886, 2)
DTIs shape =  (5383, 2)
DTIs shape =  (6385, 2)
DTIs shape =  (7371, 2)


In [9]:
nr_pfam = read_pfam(pwd = '/home/chujunyi/2_Program/2_output_file/4_target_feature/', target = 'NR')
gpcr_pfam = read_pfam(pwd = '/home/chujunyi/2_Program/2_output_file/4_target_feature/', target = 'GPCR')
ic_pfam = read_pfam(pwd = '/home/chujunyi/2_Program/2_output_file/4_target_feature/', target = 'IC')
e_pfam = read_pfam(pwd = '/home/chujunyi/2_Program/2_output_file/4_target_feature/', target = 'E')

PFAM:  (33, 31)
PFAM:  (166, 62)
PFAM:  (237, 1405)
PFAM:  (1424, 2183)


In [10]:
ic_pfam = impute_feature_0(ic_dti, ic_pfam)
e_pfam = impute_feature_0(e_dti, e_pfam)

['hsa23630', 'hsa54795']
Impute Feature:  (239, 1405)
['hsa2593', 'hsa10924', 'hsa23564', 'hsa390956', 'hsa94009', 'hsa51601', 'hsa8942']
Impute Feature:  (1431, 2183)


In [11]:
nr_x_pfam, nr_y_pfam, nr_target_pfam, nr_drug_pfam, nr_pfam_data = generate_xy_file(nr_dti, nr_pfam, target = 'NR', pfam = True)
gpcr_x_pfam, gpcr_y_pfam, gpcr_target_pfam, gpcr_drug_pfam, gpcr_pfam_data = generate_xy_file(gpcr_dti, gpcr_pfam, target = 'GPCR', pfam = True)
ic_x_pfam, ic_y_pfam, ic_target_pfam, ic_drug_pfam, ic_pfam_data = generate_xy_file(ic_dti, ic_pfam, target = 'IC', pfam = True)
e_x_pfam, e_y_pfam, e_target_pfam, e_drug_pfam, e_pfam_data = generate_xy_file(e_dti, e_pfam, target = 'E', pfam = True)

(33, 30) (33, 541) (33,) (541,) 0
(156, 61) (156, 1680) (156,) (1680,) 0
(238, 1404) (238, 765) (238,) (765,) 0
(1411, 2182) (1411, 1777) (1411,) (1777,) 0


# 合并

In [12]:
def combine_profeat_pfam(profeat_data, pfam_data, drug_profeat, target = 'NR', ctd_pfam = False, pro_pfam = False):
    profeat_feat_col = profeat_data.columns.tolist()[:-len(drug_profeat)]
    profeat_pfam = pd.merge(profeat_data[profeat_feat_col], pfam_data, left_on = 'Feature', right_on = 'Query')
    del profeat_pfam['Query']

    x = profeat_pfam.iloc[:, 1:-len(drug_profeat)]
    y = profeat_pfam.iloc[:, -len(drug_profeat):]
    target_ids = np.array(profeat_pfam['Feature'].tolist())
    drug_ids = np.array(y.columns.tolist())
    
    x_d_np = np.array(x)
    y_d_np = np.array(y)
    norm_idx = np.array(range(profeat_data[profeat_feat_col].shape[1] - 1))
    print(x.shape, y.shape, drug_ids.shape, target_ids.shape, len(norm_idx))

    if ctd_pfam:
        filename = '_U_D_xy_np_CTD_PFAM.npz'
    elif pro_pfam:
        filename = '_U_D_xy_np_PRO_PFAM.npz'

    pwd = '/home/chujunyi/2_Program/2_output_file/2_multilabel/1_X_Y_data/'
    np.savez_compressed(pwd + target + filename, 
                    x_d_np = x_d_np, 
                    y_d_np = y_d_np,
                    norm_idx = norm_idx,
                    target_ids = target_ids,
                    drug_ids = drug_ids)
    return x_d_np, y_d_np, target_ids, drug_ids

In [13]:
nr_x_ctd_pfam, nr_y_ctd_pfam, nr_target_ctd_pfam, nr_drug_ctd_pfam = combine_profeat_pfam(nr_ctd_data, nr_pfam_data, nr_drug_ctd, target = 'NR', ctd_pfam = True)
gpcr_x_ctd_pfam, gpcr_y_ctd_pfam, gpcr_target_ctd_pfam, gpcr_drug_ctd_pfam = combine_profeat_pfam(gpcr_ctd_data, gpcr_pfam_data, gpcr_drug_ctd, target = 'GPCR', ctd_pfam = True)
ic_x_ctd_pfam, ic_y_ctd_pfam, ic_target_ctd_pfam, ic_drug_ctd_pfam = combine_profeat_pfam(ic_ctd_data, ic_pfam_data, ic_drug_ctd, target = 'IC', ctd_pfam = True)
e_x_ctd_pfam, e_y_ctd_pfam, e_target_ctd_pfam, e_drug_ctd_pfam = combine_profeat_pfam(e_ctd_data, e_pfam_data, e_drug_ctd, target = 'E', ctd_pfam = True)

(33, 534) (33, 541) (541,) (33,) 504
(156, 565) (156, 1680) (1680,) (156,) 504
(238, 1908) (238, 765) (765,) (238,) 504
(1411, 2686) (1411, 1777) (1777,) (1411,) 504


In [14]:
nr_x_pro_pfam, nr_y_pro_pfam, nr_target_pro_pfam, nr_drug_pro_pfam = combine_profeat_pfam(nr_pro_data, nr_pfam_data, nr_drug_pro, target = 'NR', pro_pfam = True)
gpcr_x_pro_pfam, gpcr_y_pro_pfam, gpcr_target_pro_pfam, gpcr_drug_pro_pfam = combine_profeat_pfam(gpcr_pro_data, gpcr_pfam_data, gpcr_drug_pro, target = 'GPCR', pro_pfam = True)
ic_x_pro_pfam, ic_y_pro_pfam, ic_target_pro_pfam, ic_drug_pro_pfam = combine_profeat_pfam(ic_pro_data, ic_pfam_data, ic_drug_pro, target = 'IC', pro_pfam = True)
e_x_pro_pfam, e_y_pro_pfam, e_target_pro_pfam, e_drug_pro_pfam = combine_profeat_pfam(e_pro_data, e_pfam_data, e_drug_pro, target = 'E', pro_pfam = True)

(33, 1467) (33, 541) (541,) (33,) 1437
(156, 1498) (156, 1680) (1680,) (156,) 1437
(238, 2841) (238, 765) (765,) (238,) 1437
(1411, 3619) (1411, 1777) (1777,) (1411,) 1437


# 看一下样本有多少拥有相同的特征

In [32]:
def find_same_fps_id(fps_with_id, dti_data, hsa_col_name = 'Query'):
    duplicate = fps_with_id[fps_with_id.iloc[:, 1:].duplicated(keep = False) == True]
    duplicate = duplicate.sort_values(list(duplicate.columns[1:]))
    print('Duplicate fps = {}, ratio dup/all = {}'.format(duplicate.shape, duplicate.shape[0]/fps_with_id.shape[0]), end = ',')
    
    duplicate = duplicate.groupby(list(duplicate.columns[1:])).sum()
#     print(duplicate.columns)
    duplicate_dict = {}
    if duplicate.shape[0] != 0:
        for idx, ids in enumerate(duplicate[hsa_col_name]):
            ids = ids.replace('hsa', ',hsa')[1:].split(',')
            duplicate_dict[idx] = {}
            for id_ in ids:
                duplicate_dict[idx][id_] = list(dti_data[dti_data['hsa_id'] == id_]['drug_id'])
    print(len(duplicate_dict))
    return duplicate_dict

In [34]:
nr_pfam_duplicate_target_dict = find_same_fps_id(nr_pfam, nr_dti, 'Query')
nr_ctd_duplicate_target_dict = find_same_fps_id(nr_ctd, nr_dti, 'Feature')
nr_pro_duplicate_target_dict = find_same_fps_id(nr_pro, nr_dti, 'Feature')
# nr_ctd_pfam_duplicate_target_dict = find_same_fps_id(nr_ctd_pfam, nr_dti, 'Feature')
# nr_pro_pfam_duplicate_target_dict = find_same_fps_id(nr_pro_pfam, nr_dti, 'Feature')

Duplicate fps = (23, 31), ratio dup/all = 0.696969696969697,2
Duplicate fps = (0, 505), ratio dup/all = 0.0,0
Duplicate fps = (0, 1438), ratio dup/all = 0.0,0


In [35]:
gpcr_pfam_duplicate_target_dict = find_same_fps_id(gpcr_pfam, gpcr_dti, 'Query')
gpcr_ctd_duplicate_target_dict = find_same_fps_id(gpcr_ctd, gpcr_dti, 'Feature')
gpcr_pro_duplicate_target_dict = find_same_fps_id(gpcr_pro, gpcr_dti, 'Feature')

Duplicate fps = (148, 62), ratio dup/all = 0.891566265060241,14
Duplicate fps = (0, 505), ratio dup/all = 0.0,0
Duplicate fps = (0, 1438), ratio dup/all = 0.0,0


In [36]:
ic_pfam_duplicate_target_dict = find_same_fps_id(ic_pfam, ic_dti, 'Query')
ic_ctd_duplicate_target_dict = find_same_fps_id(ic_ctd, ic_dti, 'Feature')
ic_pro_duplicate_target_dict = find_same_fps_id(ic_pro, ic_dti, 'Feature')

Duplicate fps = (119, 1405), ratio dup/all = 0.497907949790795,22
Duplicate fps = (0, 505), ratio dup/all = 0.0,0
Duplicate fps = (0, 1438), ratio dup/all = 0.0,0


In [37]:
e_pfam_duplicate_target_dict = find_same_fps_id(e_pfam, e_dti, 'Query')
e_ctd_duplicate_target_dict = find_same_fps_id(e_ctd, e_dti, 'Feature')
e_pro_duplicate_target_dict = find_same_fps_id(e_pro, e_dti, 'Feature')

Duplicate fps = (676, 2183), ratio dup/all = 0.4723969252271139,199
Duplicate fps = (13, 505), ratio dup/all = 0.009084556254367574,6
Duplicate fps = (13, 1438), ratio dup/all = 0.009084556254367574,6


In [38]:
e_pfam_duplicate_target_dict

{0: {'hsa10924': ['D00528', 'D00501', 'D00417'],
  'hsa23564': ['D07706'],
  'hsa2593': ['DB00536', 'DB00148'],
  'hsa390956': ['D00107'],
  'hsa51601': ['D00086'],
  'hsa8942': ['D00006', 'D00012'],
  'hsa94009': ['D00160', 'D00043']},
 1: {'hsa3717': ['D10308',
   'D09783',
   'D09347',
   'D11599',
   'D09959',
   'D11296',
   'D11046',
   'D11600',
   'D10630',
   'D10358',
   'D10721',
   'D11676',
   'D09970',
   'D03003',
   'D10315',
   'D10653',
   'D11677',
   'D01441',
   'D10365',
   'D03004',
   'D09960'],
  'hsa7297': ['D09783', 'D11537', 'D09347', 'D11538', 'D09970', 'D01441']},
 2: {'hsa142679': ['D00107', 'D00184'],
  'hsa150290': ['D00107', 'D00184'],
  'hsa1847': ['D00107', 'D00184'],
  'hsa1852': ['D00107', 'D00184']},
 3: {'hsa6820': ['D00143', 'D08409'], 'hsa9955': ['D00037']},
 4: {'hsa1849': ['D00107', 'D00184'], 'hsa1850': ['D00107', 'D00184']},
 5: {'hsa84171': ['D00270'], 'hsa84695': ['D00270']},
 6: {'hsa5836': ['D00006', 'D04537', 'D02769'],
  'hsa5837': ['